In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from elastic_rods import EnergyType
from bending_validation import suppress_stdout as so

In [ ]:
import numpy as np

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
curr_um.targetDeploymentHeight = thickness * 1

In [ ]:
curr_um.energy(UmbrellaEnergyType.Elastic)

In [ ]:
curr_um.set_design_parameter_config(True, False)

In [ ]:
# curr_um.gradient(variableDesignParameters = True, designParameterOnly = False)

### Parameters

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 2
curr_um.targetDeploymentHeight = thickness * 1

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
view.show()

In [ ]:
curr_um.hessian(variableDesignParameters = True)

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
curr_um.hessianSparsityPattern()

In [ ]:
curr_um.hessian()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] += 0.5
curr_um.setDoFs(dof)

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-7
OPTS.verbose = 20
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
view.update(scalarField=rod_colors)

In [ ]:
def eqm_callback(prob, i):
    if (i % 20 == 0):
        view.update(scalarField = rod_colors)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment()

In [ ]:
from time import sleep
# sleep(2)
umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars)

In [ ]:
preoptimize_stress = curr_um.maxBendingStresses()

In [ ]:
import visualization_helper
import importlib
importlib.reload(visualization_helper)

In [ ]:
pre_sf = visualization_helper.get_scalar_field(curr_um, preoptimize_stress, rangeMin, rangeMax)

In [ ]:
view.update(scalarField=pre_sf)

In [ ]:
import umbrella_finite_diff
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, etype=elastic_rods.EnergyType.Full)

In [ ]:
view.setCameraParams(camParams)

In [ ]:
def callback(prob, i):
    if (i % 5) != 0: return
    view.update()

In [ ]:
dpo = umbrella_mesh.get_designParameter_optimizer(curr_um, callback = callback, E0 = -1, fixedVars = fixedVars)

In [ ]:
dpo.options.niter = 1000

In [ ]:
cr = dpo.optimize()

In [ ]:
postoptimize_stress = curr_um.maxBendingStresses()
post_sf = visualization_helper.get_scalar_field(curr_um, postoptimize_stress, rangeMin, rangeMax)

In [ ]:
second_postoptimize_stress = curr_um.maxBendingStresses()
second_post_sf = visualization_helper.get_scalar_field(curr_um, second_postoptimize_stress, rangeMin, rangeMax)

In [ ]:
np.max(second_postoptimize_stress)

In [ ]:
np.max(postoptimize_stress)

In [ ]:
np.max(preoptimize_stress)

In [ ]:
view.update(scalarField=second_post_sf)

In [ ]:
rangeMin = np.stack([preoptimize_stress, postoptimize_stress]).min()

In [ ]:
rangeMax = np.stack([preoptimize_stress, postoptimize_stress]).max()

In [ ]:
camParams = view.getCameraParams()